## Importamos las librerias necesarias para este ETL

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Leemos el DF con pandas
df_alcohol = pd.read_csv('BBDD Alcohol.csv')

In [3]:
# Un vistazo general del DF
df_alcohol.head()

,caso,FILTRO_A,FILTRO_B,FILTRO_C,FILTRO_C2,NSE,NSE2,P1,P2,LIMAS,...,P34_4,P34_5,P34_6,P34_7,P34_8,P34_NR,p159v1,vpond,VPOND1,VPOND2
0,1,2,1,39,4,5,C1,1,32,2,...,,3,,,1,,7,1,1.297614,1.111111
1,2,2,1,41,4,5,C1,1,32,2,...,,1 botella,,,,,8,1,1.297614,1.111111
2,3,2,1,43,4,5,C1,1,32,2,...,,1,,,,,9,1,1.297614,1.111111
3,4,2,1,51,5,4,B2,1,10,2,...,,,,1,,,10,1,0.893146,0.933333
4,5,2,1,51,5,5,C1,2,1,1,...,,1,,1,,,11,1,1.183471,1.230769


In [4]:
# Revisamos las dimensiones del DF
df_alcohol.shape

(300, 242)

In [5]:
# Filtramos el DF con los registros que cumplan con el filtro que permite avanzar en la encuesta.
df_filtrado = df_alcohol[
    (df_alcohol['FILTRO_A'] == 2) & 
    (df_alcohol['FILTRO_B'] == 1) & 
    (df_alcohol['FILTRO_C'] > 18) & 
    (df_alcohol['FILTRO_C'] < 65) & 
    (df_alcohol['FILTRO_C2'] > 1) & 
    (df_alcohol['FILTRO_C2'] < 7) & 
    (df_alcohol['NSE'] > 0) & 
    (df_alcohol['NSE'] < 5)
]

In [6]:
# Revisamos como quedo el DF luego del Filtro
df_filtrado.shape

(91, 242)

In [7]:
# Revisamos como esta distribuido el tipo de dato en las columnas
print(df_filtrado.dtypes.value_counts())

object     191
int64       49
float64      2
Name: count, dtype: int64


In [8]:
# Reemplazamos valores vacíos con valor nulo
df_filtrado = df_filtrado.replace([' '], pd.NA)

In [9]:
# Verificamos los cambios de vacios a Nulos
df_filtrado

,caso,FILTRO_A,FILTRO_B,FILTRO_C,FILTRO_C2,NSE,NSE2,P1,P2,LIMAS,...,P34_4,P34_5,P34_6,P34_7,P34_8,P34_NR,p159v1,vpond,VPOND1,VPOND2
3,4,2,1,51,5,4,B2,1,10,2,...,<NA>,<NA>,<NA>,1,<NA>,<NA>,10,1,0.893146,0.933333
7,8,2,1,24,2,4,B2,1,17,3,...,<NA>,3,<NA>,1,<NA>,<NA>,15,1,0.893146,0.933333
15,16,2,1,31,3,3,B1,2,13,1,...,<NA>,1,<NA>,1,<NA>,<NA>,23,1,0.847847,1.055556
17,18,2,1,32,3,3,B1,2,33,4,...,<NA>,1,<NA>,1,<NA>,<NA>,25,1,1.990974,1.400000
18,19,2,1,40,4,2,A2,2,1,1,...,<NA>,1 botella,<NA>,1 botellita,<NA>,<NA>,26,1,0.714107,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,262,2,1,43,4,4,B2,2,36,1,...,1,1,1,1,<NA>,<NA>,285,1,0.847847,1.055556
276,277,2,1,21,2,3,B1,1,46,5,...,<NA>,1,<NA>,<NA>,<NA>,<NA>,301,1,1.109755,1.000000
278,279,2,1,38,4,4,B2,1,46,5,...,<NA>,1,<NA>,<NA>,<NA>,<NA>,303,1,1.109755,1.000000
281,282,2,1,44,4,4,B2,1,32,2,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,306,1,0.818151,0.714286


In [10]:
def extraer_numeros(celda):
    if isinstance(celda, str):  # Verifica si el valor es una cadena de texto
        numeros = re.findall(r'\d+', celda)
        numero = ''.join(numeros)
        return pd.to_numeric(numero, errors='coerce')  # Convierte a numérico o NaN
    return pd.to_numeric(celda, errors='coerce')  # Asegura que todos los valores sean numéricos o NaN

# Lista de columnas que no deben ser tocadas
columnas_no_modificar = ['NSE2', 'VPOND1', 'VPOND2']

# Aplica la función a las columnas que no están en la lista de exclusión
for columna in df_filtrado.columns:
    if columna not in columnas_no_modificar:
        df_filtrado[columna] = df_filtrado[columna].apply(extraer_numeros)

# Convertir las columnas modificadas a Int64
for columna in df_filtrado.columns:
    if columna not in columnas_no_modificar:
        df_filtrado[columna] = df_filtrado[columna].astype('Int64')

In [11]:
df_filtrado

,caso,FILTRO_A,FILTRO_B,FILTRO_C,FILTRO_C2,NSE,NSE2,P1,P2,LIMAS,...,P34_4,P34_5,P34_6,P34_7,P34_8,P34_NR,p159v1,vpond,VPOND1,VPOND2
3,4,2,1,51,5,4,B2,1,10,2,...,<NA>,<NA>,<NA>,1,<NA>,<NA>,10,1,0.893146,0.933333
7,8,2,1,24,2,4,B2,1,17,3,...,<NA>,3,<NA>,1,<NA>,<NA>,15,1,0.893146,0.933333
15,16,2,1,31,3,3,B1,2,13,1,...,<NA>,1,<NA>,1,<NA>,<NA>,23,1,0.847847,1.055556
17,18,2,1,32,3,3,B1,2,33,4,...,<NA>,1,<NA>,1,<NA>,<NA>,25,1,1.990974,1.400000
18,19,2,1,40,4,2,A2,2,1,1,...,<NA>,1,<NA>,1,<NA>,<NA>,26,1,0.714107,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,262,2,1,43,4,4,B2,2,36,1,...,1,1,1,1,<NA>,<NA>,285,1,0.847847,1.055556
276,277,2,1,21,2,3,B1,1,46,5,...,<NA>,1,<NA>,<NA>,<NA>,<NA>,301,1,1.109755,1.000000
278,279,2,1,38,4,4,B2,1,46,5,...,<NA>,1,<NA>,<NA>,<NA>,<NA>,303,1,1.109755,1.000000
281,282,2,1,44,4,4,B2,1,32,2,...,<NA>,2,<NA>,<NA>,<NA>,<NA>,306,1,0.818151,0.714286


In [12]:
# Revisamos la distribución de tipos de columnas luego de los cambios
print(df_filtrado.dtypes.value_counts())

Int64      239
float64      2
object       1
Name: count, dtype: int64


In [13]:
df_filtrado.shape

(91, 242)

In [14]:
# Filtro de datos del perfil, para evitar outliers
df_filtro_2 = df_filtrado.loc[
    (df_filtrado['P1'].isin([1, 2])) &
    (df_filtrado['P2'] > 0) & (df_filtrado['P2'] < 50) &
    (df_filtrado['P3'] > 0) & (df_filtrado['P3'] < 9)
]

In [15]:
df_filtro_2.shape

(91, 242)

In [16]:
columnas_con_nulos = df_filtro_2.isnull().sum()
columnas_con_nulos = columnas_con_nulos[columnas_con_nulos > 0]
print(columnas_con_nulos)

P5_2      11
P5_3      38
P5_4      66
P5_5      85
P6_2      59
          ..
P34_5     21
P34_6     87
P34_7     40
P34_8     87
P34_NR    91
Length: 190, dtype: int64


In [23]:
# Contar la cantidad de columnas con todos sus registros nulos
num_columnas_nulos_totales = sum(df_filtro_2.isnull().sum() == len(df_filtro_2))
print("Cantidad de columnas con todos sus registros nulos:", num_columnas_nulos_totales)

Cantidad de columnas con todos sus registros nulos: 18


In [22]:
# Filtrar columnas con todos los registros nulos
columnas_nulos_totales = df_filtro_2.columns[df_filtro_2.isnull().all()]
# Convertir el resultado a una lista de nombres de columnas
nombres_columnas_nulos_totales = list(columnas_nulos_totales)

print("Nombres de las columnas con todos sus registros nulos:", nombres_columnas_nulos_totales)

Nombres de las columnas con todos sus registros nulos: ['P7_4_2', 'P7_4_3', 'P7_5_4', 'NPS_Coderal', 'P23_3', 'P24_5_4', 'P24_5_5', 'P24_8_4', 'P24_12_4', 'P27_8', 'P28_5', 'P28_6', 'P32_3_2', 'P32_6_2', 'P32_7_3', 'P32_NR', 'P33_NR', 'P34_NR']


In [24]:
# Cálculo porcentaje de valores nulos en cada columna
porcentaje_nulos_por_columna = (df_filtro_2.isnull().sum() / len(df_filtro_2)) * 100
# Contar cuántas columnas tienen más del 50% de registros nulos
columnas_mas_50_nulos = porcentaje_nulos_por_columna[porcentaje_nulos_por_columna > 50]
cantidad_columnas_mas_50_nulos = len(columnas_mas_50_nulos)

print("Cantidad de columnas con más del 50% de registros nulos:", cantidad_columnas_mas_50_nulos)


Cantidad de columnas con más del 50% de registros nulos: 160


In [26]:
# Calcular el porcentaje de valores nulos en cada fila
porcentaje_nulos_por_fila = (df_filtro_2.isnull().sum(axis=1) / len(df_filtro_2.columns)) * 100

# Contar cuántas filas tienen más del 50% de valores nulos
filas_mas_50_nulos = porcentaje_nulos_por_fila[porcentaje_nulos_por_fila > 50]

cantidad_filas_mas_50_nulos = len(filas_mas_50_nulos)

print("Cantidad de filas con más del 50% de valores nulos:", cantidad_filas_mas_50_nulos)

Cantidad de filas con más del 50% de valores nulos: 87
